### Data extraction
Extract data from MIMIC-III

In [1]:
import time
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import cufflinks

cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')

In [2]:
# Import pandas 
import pandas as pd 

start = time.time()
# reading csv file 
admissions_df = pd.read_csv("../MIMIC-III/ADMISSIONS.csv")
noteevents_df = pd.read_csv("../MIMIC-III/NOTEEVENTS.csv") 
end = time.time()
print(end - start)


/data/home/evgenii/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning:

Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.



70.83860993385315


In [3]:
start = time.time()
# Left join of two dataframes
note_admiss_df_left = noteevents_df.merge(admissions_df, on='HADM_ID', how='left', indicator=True)
end = time.time()
print(end - start)

5.864101409912109


In [4]:
note_admiss_df_left[['DIAGNOSIS', 'SUBJECT_ID_x', 'SUBJECT_ID_y','DESCRIPTION', 'CATEGORY']]

,DIAGNOSIS,SUBJECT_ID_x,SUBJECT_ID_y,DESCRIPTION,CATEGORY
0,RT LOWER LOBE PNEUMONIA,22532,22532.0,Report,Discharge summary
1,CHRONIC OBSTRUCTIVE PULMONARY DISEASE,13702,13702.0,Report,Discharge summary
2,TRACHEALBRONCHEAL MALACEA/SDA,13702,13702.0,Report,Discharge summary
3,ASTHMA;COPD EXACERBATION,13702,13702.0,Report,Discharge summary
4,S/P FALL;TELEMETRY,26880,26880.0,Report,Discharge summary
...,...,...,...,...,...
2083175,NEWBORN,31097,31097.0,Report,Nursing/other
2083176,NEWBORN,31097,31097.0,Report,Nursing/other
2083177,NEWBORN,31097,31097.0,Report,Nursing/other
2083178,NEWBORN,31097,31097.0,Report,Nursing/other


In [5]:
pneumonia_df = note_admiss_df_left.loc[note_admiss_df_left["DIAGNOSIS"] == 'PNEUMONIA', ['ROW_ID_x', 'SUBJECT_ID_x', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'DIAGNOSIS', 'HAS_CHARTEVENTS_DATA']]

In [6]:
#pneumonia_no_disch_df = pneumonia_df.loc[pneumonia_df["CATEGORY"] != 'Discharge summary', ['ROW_ID_x', 'SUBJECT_ID_x', 'HADM_ID', 'CHARTDATE', 'CHARTTIME', 'STORETIME', 'CATEGORY', 'DESCRIPTION', 'CGID', 'ISERROR', 'TEXT', 'ADMITTIME', 'DISCHTIME', 'DEATHTIME', 'ADMISSION_TYPE', 'DIAGNOSIS', 'HAS_CHARTEVENTS_DATA']]
pneumonia_no_disch_df = pneumonia_df.loc[pneumonia_df["CATEGORY"] != 'Discharge summary', ['ROW_ID_x','SUBJECT_ID_x','CHARTDATE','STORETIME','CATEGORY','DESCRIPTION','TEXT', 'DEATHTIME']]

In [7]:
pneumonia_alive_no_disch = pneumonia_no_disch_df[pneumonia_no_disch_df.DEATHTIME.isnull()]
pneumonia_dead_no_disch = pneumonia_no_disch_df[pneumonia_no_disch_df.DEATHTIME.notnull()]

In [8]:
pneumonia_dead_no_disch.shape

(11895, 8)

In [9]:
pneumonia_alive_no_disch.shape

(46134, 8)

In [10]:
pneumonia_alive_no_disch = pneumonia_alive_no_disch.sort_values(by=['SUBJECT_ID_x','CHARTDATE', 'ROW_ID_x'])
pneumonia_dead_no_disch = pneumonia_dead_no_disch.sort_values(by=['SUBJECT_ID_x','CHARTDATE', 'ROW_ID_x'])

In [11]:
# Number of patients
pneumonia_dead_no_disch['SUBJECT_ID_x'].nunique()

252

In [12]:
# Number of patients
pneumonia_alive_no_disch['SUBJECT_ID_x'].nunique()

1167

In [13]:
# import text handling tool
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from collections import Counter

import sys
import re
import csv
import os.path

# define constants
# RESULT_HEADER = "WORD, FREQUENCY\n"
MIN_SEQ_LEN = 4
USE_1_N_SEQ = 2


# words that do not have meaning (can be modified later)
USELESS_WORDS = ["a", "the", "he", "she", ",", ".", "?", "!", ":", ";", "+", "*", "**"\
                 "your", "you"]

# count up the frequency of every word in every disease file
stemmer = PorterStemmer()
# create set of words to ignore in text
stop_words = set(stopwords.words('english'))

for word in USELESS_WORDS:
    stop_words.add(word)

In [14]:

#----------
def count_notes_per_patient(disease_df):
    patient_id_to_num_notes = {}
    patient_id = -1
    note_counter = 0
            
    for index, row in tqdm(disease_df.iterrows(), total=disease_df.shape[0]):
        patient_id_check = int(row['SUBJECT_ID_x'])
                
        if not patient_id == patient_id_check:
            patient_id_to_num_notes[patient_id] = note_counter
            note_counter = 1
        else:
            note_counter += 1
                    
        patient_id = patient_id_check
                
    patient_id_to_num_notes[patient_id] = note_counter
    del patient_id_to_num_notes[-1]
    return patient_id_to_num_notes

patient_id_to_num_notes = {}
patient_id_to_num_notes['pneumonia_dead'] = count_notes_per_patient(pneumonia_dead_no_disch)
patient_id_to_num_notes['pneumonia_alive'] = count_notes_per_patient(pneumonia_alive_no_disch)

100%|██████████| 46134/46134 [00:05<00:00, 9199.37it/s]


In [15]:
def count_words_per_patient(disease_df, patient_id_to_num_notes):
    note_appearance_counter = {}
    number_of_patients = 0 # number of patients
    note_counter = 0

# -----------
    patient_id = -1
    word_set = set()
    note_event_counter = 0

    # Iterate through each note
    for index, row in tqdm(disease_df.iterrows(), total=disease_df.shape[0]):

        
        patient_id_check = int(row['SUBJECT_ID_x'])
    
        # if patient id has changed, end sequence and start new sequence
        if not patient_id == patient_id_check:
            number_of_patients += 1
            note_event_counter = 0
        
            for word in word_set:
                if word in note_appearance_counter:
                    note_appearance_counter[word] += 1
                else:
                    note_appearance_counter[word] = 1

        
            # reset word_set
            word_set = set()
        
        # update patient id
        patient_id = patient_id_check

            
        if patient_id_to_num_notes[patient_id_check] <= MIN_SEQ_LEN:
            continue
            
        if note_event_counter < patient_id_to_num_notes[patient_id] // USE_1_N_SEQ:
            note_event_counter += 1
            continue
                
        note_counter += 1
        note = re.sub(r'\[\*\*(.*?)\*\*\]|[_,\d\*:~=\.\-\+\\/]+', ' ', row['TEXT'])
        tokenized_note = word_tokenize(note)
        
        
        for word in tokenized_note:
            stemmed_word = stemmer.stem(word.lower())
            if not stemmed_word in stop_words:
                word_set.add(stemmed_word)
    
    print(str(note_counter) + " note events")
    print("finished counting frequent words for patients!")
#     return note_counter, note_appearance_counter
    return number_of_patients, note_appearance_counter

# variable dictionaries
number_of_notes = {}
note_appearance_counter = {}

number_of_notes['pneumonia_dead'], note_appearance_counter['pneumonia_dead'] = count_words_per_patient(pneumonia_dead_no_disch, patient_id_to_num_notes['pneumonia_dead'])
number_of_notes['pneumonia_alive'], note_appearance_counter['pneumonia_alive'] = count_words_per_patient(pneumonia_alive_no_disch, patient_id_to_num_notes['pneumonia_alive'])

  0%|          | 22/46134 [00:00<03:29, 219.76it/s]

5989 note events
finished counting frequent words for patients!


100%|██████████| 46134/46134 [02:59<00:00, 257.67it/s]

23172 note events
finished counting frequent words for patients!


In [16]:
# # get hyper-paramters n_fold and threshold from user input
n_fold = float(3)
threshold = float(0.01)

frequent_word_lists = {}
factor = {}

In [17]:
"""function description:
for each disease in note_appearance_counter
    1. checks whether a word in disease file is frequent(frequency standard as defined by factor, n_fold, and threshold)
    2. adds to frequent_word_list
"""

# def find_frequent_word(note_appearance_counter, frequent_word_lists, number_of_notes, factor, n_fold, threshold):

# calculate normalizing factor for each disease
note_sum = 0

# Count from two labels
for disease in number_of_notes:
    note_sum += float(number_of_notes[disease])
    
for disease in number_of_notes:
    factor[disease] = number_of_notes[disease] / note_sum

# determine frequent word for each disease file
for disease in note_appearance_counter:
    frequent_word_lists[disease] = []

    print(disease + " has " + str(len(note_appearance_counter[disease])) + " unique words!")

    for word in note_appearance_counter[disease]:
        
        freq_check = True
        for check_disease in note_appearance_counter:
            
            if not disease == check_disease:
                if word in note_appearance_counter[check_disease]:
                    if not (note_appearance_counter[disease][word] / note_appearance_counter[check_disease][word] / factor[disease] * factor[check_disease] > n_fold \
                        and note_appearance_counter[disease][word] > (number_of_notes[disease] * threshold)):

                        freq_check = False
                        break

                else:
                    if not (note_appearance_counter[disease][word] > n_fold and note_appearance_counter[disease][word] > (number_of_notes[disease] * threshold)):
                        freq_check = False
                        break
        if freq_check:
            frequent_word_lists[disease].append((word))
            # Create a tuple of word and its count
#             frequent_word_lists[disease].append((word, note_appearance_counter[disease][word]))

        

print("finished making frequent words list for " + disease)

pneumonia_dead has 11678 unique words!
pneumonia_alive has 22572 unique words!
finished making frequent words list for pneumonia_alive


In [18]:
print(len(frequent_word_lists['pneumonia_dead']))
print(len(frequent_word_lists['pneumonia_alive']))

454
329


### 2. Co-occurrence generation

In [19]:
FREQUENT_WORD_LIST = frequent_word_lists['pneumonia_dead'] + frequent_word_lists['pneumonia_alive']
print(len(FREQUENT_WORD_LIST))
print(len(set(FREQUENT_WORD_LIST)))

783
783


In [20]:
"""function description:
generates frequent word set for the disease
"""
word_dict = {}
word_id = 1
stemmer = PorterStemmer()


for word in FREQUENT_WORD_LIST:
    if not word == "WORD":
        word_dict[stemmer.stem(word.strip())] = word_id
        word_id += 1
             
print(f"\nword dictionary created! Length: {len(word_dict)}\n")
print(word_dict)


word dictionary created! Length: 783

{'cremat': 1, 'vasopressin': 2, 'anur': 3, 'phenylephrin': 4, 'bicarbon': 5, 'mottl': 6, 'cpr': 7, 'arrest': 8, 'clinician': 9, 'arrang': 10, 'unrespon': 11, 'pea': 12, 'park': 13, 'hypercapnia': 14, 'refractori': 15, 'elig': 16, 'norepinephrin': 17, 'profound': 18, 'dic': 19, 'azotemia': 20, 'hypothermia': 21, 'acidem': 22, 'hypercapn': 23, 'funer': 24, 'paralyt': 25, 'suffer': 26, 'death': 27, 'autopeep': 28, 'clergi': 29, 'recruit': 30, 'expir': 31, 'pcv': 32, 'slough': 33, 'lubric': 34, 'gum': 35, 'sluggish': 36, 'dnr': 37, 'nacl': 38, 'cmo': 39, 'pinpoint': 40, 'virtual': 41, 'dk': 42, 'dni': 43, 'nonpurpo': 44, 'oder': 45, 'float': 46, 'mso': 47, 'clariti': 48, 'levoph': 49, 'wish': 50, 'institut': 51, 'pronounc': 52, 'option': 53, 'grim': 54, 'bow': 55, 'hypot': 56, 'respon': 57, 'prognosi': 58, 'purpl': 59, 'dd': 60, 'remin': 61, 'suport': 62, 'dead': 63, 'peerla': 64, 'gather': 65, 'viabl': 66, 'priest': 67, 'antb': 68, 'asystol': 69, 'ac

### Count co-occurrences per patient

In [ ]:
# Comparison algorithms
# Apriori, Fpgrowth, fp_max

# Apriori
import pandas as pd

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import fpgrowth

# ----
def apr_patterns_per_patient_python(disease_df, min_support):
    
    # For dataframe
    patient_cooc_dict = {}
    patient_node_dict = {}
    patient_note_cnt = {}
    patient_itemset_dict = {}
    
    # --------------    
    patient_id = -1
    note_cnt = 0
    patient_note_list = []
    
    # read line in from file (each line is one note)
    for index, row in tqdm(disease_df.iterrows(), total=disease_df.shape[0]):
        
        # only regard certain type of notes
        patient_id_check = int(row['SUBJECT_ID_x'])
        note = re.sub(r'\[\*\*(.*?)\*\*\]|[_,\d\*:~=\.\-\+\\/]+', ' ', row['TEXT'])
        patient_word_set = set()
    
#         print(f"patient_id_check: {patient_id_check}, patient_id: {patient_id}")
        # if patient id has changed, end sequence and start new sequence
        if not patient_id == patient_id_check and not patient_id == -1:
            te = TransactionEncoder()
            te_ary = te.fit(patient_note_list).transform(patient_note_list)
            df = pd.DataFrame(te_ary, columns=te.columns_)
            df_itemsets = fpgrowth(df, min_support=min_support, use_colnames=True)
            
            cooc_tmp = []
            cooc_minsup_tmp = []
            node_minsup_tmp = []
            cooc_node_idx_tmp = []
            itemsets_tmp = []
            
            for index, row in df_itemsets.iterrows():
                if len(row['itemsets']) == 1:
                    word = list(row['itemsets'])[0]
                    min_sup = row['support']

                    node_minsup_tmp.append(len(patient_note_list) * row['support'])
                    cooc_node_idx_tmp.append(word)
                    
                if len(row['itemsets']) == 2:
                    cooc_ = sorted(list(row['itemsets']))
                    cooc_tmp.append(cooc_)
                    cooc_minsup_tmp.append(len(patient_note_list) * row['support'])

                if len(row['itemsets']) >= 2:
                    itemset = sorted(list(row['itemsets']))
                    itemsets_tmp.append(itemset)
                    
            
            cooc_dict = {}

            for num, i in enumerate(cooc_tmp):
                if tuple(i) not in cooc_dict:
                    cooc_dict[tuple(i)] = cooc_minsup_tmp[num]


            node_dict = {}
            for num, i in enumerate(cooc_node_idx_tmp):
                if i not in node_dict:
                    node_dict[i] = node_minsup_tmp[num]

            # Update glob lists
            if patient_id not in patient_cooc_dict:
                patient_cooc_dict[patient_id] = cooc_dict
                patient_node_dict[patient_id] = node_dict
                patient_note_cnt[patient_id] = note_cnt
                patient_itemset_dict[patient_id] = itemsets_tmp
    
            else:
                print(f"patient_id: {patient_id} is already in the dictionary!")
            
            
            # Reset local lists
            patient_note_list = []
            note_cnt = 0
                    
        # update patient id
        patient_id = patient_id_check
        tokenized_note = word_tokenize(note)
        note_cnt += 1

        # loop through each word in note to count word belonging to each disease
        for word in tokenized_note:
            stemmed_word = stemmer.stem(word.lower())       
#             if stemmed_word in word_dict:
            patient_word_set.add(stemmed_word)

        templst = []
        for word in patient_word_set:
            templst.append(word)

        if templst:
            patient_note_list.append(templst)
    
    # Last patient info
    te = TransactionEncoder()
    te_ary = te.fit(patient_note_list).transform(patient_note_list)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    df_itemsets = fpgrowth(df, min_support=min_support, use_colnames=True)
            
    cooc_tmp = []
    cooc_minsup_tmp = []
    #-----
    node_minsup_tmp = []
    cooc_node_idx_tmp = []
    itemsets_tmp = []
            
    for index, row in df_itemsets.iterrows():
        if len(row['itemsets']) == 1:
            word = list(row['itemsets'])[0]
            node_minsup_tmp.append(len(patient_note_list) * row['support'])
            cooc_node_idx_tmp.append(word)

        if len(row['itemsets']) == 2:
            cooc_ = sorted(list(row['itemsets']))
            cooc_tmp.append(cooc_)
            cooc_minsup_tmp.append(len(patient_note_list) * row['support'])

        if len(row['itemsets']) >= 2:
            itemset = sorted(list(row['itemsets']))
            itemsets_tmp.append(itemset)

    cooc_dict = {}

    for num, i in enumerate(cooc_tmp):
        if tuple(i) not in cooc_dict:
            cooc_dict[tuple(i)] = cooc_minsup_tmp[num]


    node_dict = {}
    for num, i in enumerate(cooc_node_idx_tmp):
        if i not in node_dict:
            node_dict[i] = node_minsup_tmp[num]
    
                

    # Update glob lists
    if patient_id not in patient_cooc_dict:
        patient_cooc_dict[patient_id] = cooc_dict
        patient_node_dict[patient_id] = node_dict
        patient_note_cnt[patient_id] = note_cnt
        patient_itemset_dict[patient_id] = itemsets_tmp
    else:
        print(f"patient_id: {patient_id} is already in the dictionary!")

    return patient_node_dict, patient_cooc_dict, patient_itemset_dict, patient_note_cnt

apr_patient_node_0, apr_patient_cooc_0, apr_patient_itemset_0, apr_patient_note_num_0 = apr_patterns_per_patient_python(pneumonia_dead_no_disch, 0.999)
apr_patient_node_1, apr_patient_cooc_1, apr_patient_itemset_1, apr_patient_note_num_1 = apr_patterns_per_patient_python(pneumonia_alive_no_disch, 0.999)

 21%|██▏       | 9902/46134 [04:40<02:27, 245.31it/s]  

In [ ]:
# Create a set of all unique co-occurrences from both group
patient_cooc_set = set()
patient_cooc_0_dict = {}
patient_cooc_1_dict = {}

for k, v in apr_patient_cooc_0.items():
    for item in v:
        patient_cooc_set.add(item)
        if item not in patient_cooc_0_dict:
            patient_cooc_0_dict[item] = v[item]
        else:
            patient_cooc_0_dict[item] = patient_cooc_0_dict[item] + v[item]
            
    
for k, v in apr_patient_cooc_1.items():
    for item in v:
        patient_cooc_set.add(item)
        if item not in patient_cooc_1_dict:
            patient_cooc_1_dict[item] = v[item]
        else:
            patient_cooc_1_dict[item] = patient_cooc_1_dict[item] + v[item]

In [ ]:
print(len(patient_cooc_set))
print(len(patient_cooc_0_dict))
print(len(patient_cooc_1_dict))

In [ ]:
import math
patient_cooc_odd_scores = {}
for set_item in patient_cooc_set:
    if set_item in patient_cooc_0_dict and set_item in patient_cooc_1_dict: 
        d_prob = patient_cooc_0_dict[set_item]/(patient_cooc_0_dict[set_item] + patient_cooc_1_dict[set_item])
        a_prob = patient_cooc_1_dict[set_item]/(patient_cooc_0_dict[set_item] + patient_cooc_1_dict[set_item])
        log_odd_score = math.log((a_prob + 0.001)/(d_prob+0.001))
        patient_cooc_odd_scores[set_item] = log_odd_score
    elif set_item in patient_cooc_0_dict:
        log_odd_score = math.log((0.001)/(1.001))
        patient_cooc_odd_scores[set_item] = log_odd_score
    elif set_item in patient_cooc_1_dict:
        log_odd_score = math.log((1.001)/(0.001))
        patient_cooc_odd_scores[set_item] = log_odd_score

In [ ]:
patient_cooc_odd_scores

In [ ]:
# Data normalization
import numpy
def data_norm(cooc_odd_scores):

    def norm_arr(array):
        arr = numpy.array(list(array))
        start = 0
        end = 1
        width = end - start
        res = (arr - arr.min())/(arr.max() - arr.min()) * width + start
        return res.tolist()

    cooc_keys, cooc_values = zip(*cooc_odd_scores.items())
    new_cooc_odd_scores = dict(zip(cooc_keys, norm_arr(cooc_values)))

    return new_cooc_odd_scores

new_patient_cooc_odd_scores = data_norm(patient_cooc_odd_scores)

In [ ]:
# Additional code to label node words
apr_patient_node_set = set()
apr_patient_node_0_dict = {}
apr_patient_node_1_dict = {}
    
for k, v in apr_patient_node_0.items():
    for item in v:
        apr_patient_node_set.add(item)
        
        if item not in apr_patient_node_0_dict:
            apr_patient_node_0_dict[item] = v[item]
        else:
            apr_patient_node_0_dict[item] = apr_patient_node_0_dict[item] + v[item]
        
    
for k, v in apr_patient_node_1.items():
    for item in v:
        apr_patient_node_set.add(item)
        
        if item not in apr_patient_node_1_dict:
            apr_patient_node_1_dict[item] = v[item]
        else:
            apr_patient_node_1_dict[item] = apr_patient_node_1_dict[item] + v[item]

In [ ]:
print(len(apr_patient_node_set))
print(len(apr_patient_node_0_dict))
print(len(apr_patient_node_1_dict))

In [ ]:
apr_patient_node_odd_scores = {}

for set_item in apr_patient_node_set:
    if set_item in apr_patient_node_0_dict and set_item in apr_patient_node_1_dict:
        d_prob = apr_patient_node_0_dict[set_item]/(apr_patient_node_0_dict[set_item] + apr_patient_node_1_dict[set_item])
        a_prob = apr_patient_node_1_dict[set_item]/(apr_patient_node_0_dict[set_item] + apr_patient_node_1_dict[set_item])
        log_odd_score = math.log((a_prob + 0.001)/(d_prob+0.001))
        apr_patient_node_odd_scores[set_item] = log_odd_score
    elif set_item in apr_patient_node_0_dict:
        log_odd_score = math.log((0.001)/(1.001))
        apr_patient_node_odd_scores[set_item] = log_odd_score
    elif set_item in apr_patient_node_1_dict:
        log_odd_score = math.log((1.001)/(0.001))
        apr_patient_node_odd_scores[set_item] = log_odd_score

In [ ]:
print(apr_patient_node_odd_scores)

In [ ]:
apr_patient_square_node_data = pd.DataFrame({'node':list(apr_patient_node_odd_scores.keys()), 'feature':list(apr_patient_node_odd_scores.values())})
apr_patient_square_node_data

In [ ]:
apr_patient_square_node_id_data = apr_patient_square_node_data.set_index("node")
apr_patient_square_node_id_data['subject'] = ["positive" if r['feature'] > 0 else "negative" for i, r in apr_patient_square_node_id_data.iterrows()]
apr_patient_square_node_id_data

In [ ]:
apr_subjects = apr_patient_square_node_id_data["subject"]
apr_subjects

In [ ]:
# Create a list of all itemsets
apr_itemsets = []

for k,v in tqdm(apr_patient_itemset_0.items()):
    
    for i in v:  
        if i not in apr_itemsets:
            apr_itemsets.append(i)
    
for k,v in tqdm(apr_patient_itemset_1.items()):
    for i in v:
        if i not in apr_itemsets:
            apr_itemsets.append(i)

In [ ]:
print(len(apr_itemsets))

In [ ]:
print(apr_itemsets[:100])

#### Create embeddings from apr_cooc

In [ ]:
from sklearn.manifold import TSNE
from gensim.models import Word2Vec

In [ ]:
apr_cooc_model_patient = Word2Vec(
    apr_itemsets, size=128, window=5, min_count=0, sg=1, workers=4, iter=1
)

In [ ]:
emb = apr_cooc_model_patient.wv["cmo"]
emb

#### Visualise node embeddings for apr_cooc

In [ ]:
# Retrieve node embeddings and corresponding subjects
patient_node_ids = apr_cooc_model_patient.wv.index2word  # list of node IDs
patient_weighted_node_embeddings = (
    apr_cooc_model_patient.wv.vectors
)  # numpy.ndarray of size number of nodes times embeddings dimensionality
# the gensim ordering may not match the StellarGraph one, so rearrange
patient_node_targets = apr_subjects.loc[patient_node_ids].astype("category")

In [ ]:
# Apply t-SNE transformation on node embeddings
tsne = TSNE(n_components=2, random_state=42)
patient_weighted_node_embeddings_2d = tsne.fit_transform(patient_weighted_node_embeddings)

In [ ]:
# draw the points
alpha = 0.7

plt.figure(figsize=(10, 8))
plt.scatter(
    patient_weighted_node_embeddings_2d[:, 0],
    patient_weighted_node_embeddings_2d[:, 1],
    c=patient_node_targets.cat.codes,
    cmap="jet",
    alpha=0.7,
)
plt.show()

In [ ]:
# Create a random vector for words that are not in a dictionary
apr_node_emb_dict = {}
rand_vec_cnt = 0
for index, row in apr_patient_square_node_id_data.iterrows():
    try:
        if index not in apr_node_emb_dict:
            apr_node_emb_dict[index] = apr_cooc_model_patient.wv[index]
        else:
            print(f"index: {index} is already in a dictionary!")
    except KeyError:
        rand_vec_cnt += 1
        apr_node_emb_dict[index] = np.random.normal(scale=0.6, size=128)

print(f"random vector counter is {rand_vec_cnt}")

### Load the data

In [ ]:
import pandas as pd
import numpy as np

import stellargraph as sg
from stellargraph.mapper import PaddedGraphGenerator
from stellargraph.layer import GCNSupervisedGraphClassification
from stellargraph import StellarGraph

from stellargraph import datasets

from sklearn import model_selection
from IPython.display import display, HTML

from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
import matplotlib.pyplot as plt

seed = 42
numpy.random.seed(seed)
tf.random.set_seed(seed)
sg.random.set_seed(seed)

#### Train apr

In [ ]:
from collections import defaultdict

def create_apr_graph_list(patient_cooc_dict, cooc_odd_scores, node_emb_dict, label):
    
    graphs = []
    labels = []
    
    for key,row in patient_cooc_dict.items():
     
        if row:
            source = []
            target = []
            # edge_weight = []
            
            node_feature = []
            node_idx = []
            for cooc in row:
                source.extend([cooc[0], cooc[1]])
                target.extend([cooc[1], cooc[0]])
                # edge_weight.extend([cooc_odd_scores[cooc], cooc_odd_scores[cooc]])
        
            node_idx = list(set(source + target))
    
            # Create a dataframe of only nodes
            square_node_data = pd.DataFrame(
                index=node_idx)
            
#             square_node_data['feature'] = [1] * len(node_idx)
            
            # Create a dictionary for each column for a vector
            node_features = defaultdict(list)
            for node in node_idx:
                for i, vec in enumerate(node_emb_dict[node]):
                    node_features['w_' + str(i)].append(vec)
        
            # Add columns to a dataframe
            for k, v in node_features.items():
              
                square_node_data[k] = v

            square_edges = pd.DataFrame({ 
                "source": source, 
                "target": target, 
                # "weight":edge_weight
            })
        
            square = StellarGraph({"corner": square_node_data}, {"line": square_edges})
            graphs.append(square)
            labels.append(label)
            
    return graphs, labels

graphs = []
labels = []

# patient_weighted_node_emb_dict
graph_0, label_0 = create_apr_graph_list(apr_patient_cooc_0, new_patient_cooc_odd_scores, apr_node_emb_dict, -1)
graph_1, label_1 = create_apr_graph_list(apr_patient_cooc_1, new_patient_cooc_odd_scores, apr_node_emb_dict, 1)

graphs.extend(graph_0)
labels.extend(label_0)
print(f"graphs_0: {len(graphs)}, labels_0: {len(labels)}")
graphs.extend(graph_1)
labels.extend(label_1)
print(f"graphs_1: {len(graph_1)}, labels_1: {len(label_1)}")
print(f"graphs: {len(graphs)}, labels: {len(labels)}")

In [ ]:
# prepare test and train datasets
test_cnt = int(len(graphs)*0.1)/2
print(test_cnt)

pos_start = len(graph_0)
print(pos_start)

test_arr = []
train_arr = []
for i, v in enumerate(graphs):
    # Take first items for neg set
    if i < test_cnt:
        test_arr.append(i)    
    elif i > pos_start and i <= pos_start + test_cnt:
        test_arr.append(i)    
    else:
        train_arr.append(i)

In [ ]:
import random
seed = 42
c = list(test_arr)
random.Random(seed).shuffle(c)
test_arr =  c

In [ ]:
c = list(train_arr)
random.Random(seed).shuffle(c)
train_arr =  c

In [ ]:
train_index = np.array(train_arr)
test_index = np.array(test_arr)

In [ ]:
print(len(test_index))
print(test_index)

In [ ]:
print(len(train_index))
print(train_index)

In [ ]:
print(test_arr)

In [ ]:
# # shuffle the data
# import random
# seed = 42

# c = list(zip(graphs, labels))
# random.Random(seed).shuffle(c)

# graphs, labels = zip(*c)

In [ ]:
print(labels)

In [ ]:
print(graphs[175].info())

In [ ]:
print(graphs[0].node_features())
print(len(graphs[0].node_features()[0]))

In [ ]:
print(graphs[100].info())

In [ ]:
summary = pd.DataFrame(
    [(g.number_of_nodes(), g.number_of_edges()) for g in graphs],
    columns=["nodes", "edges"],
)
summary.describe().round(1)

In [ ]:
graph_labels = pd.Series(labels)

In [ ]:
print(graph_labels)

In [ ]:
graph_labels.value_counts().to_frame()

In [ ]:
graph_labels = pd.get_dummies(graph_labels, drop_first=True)

In [ ]:
print(type(graph_labels))

In [ ]:
graph_labels

In [ ]:
generator = PaddedGraphGenerator(graphs=graphs)

In [ ]:
def create_graph_classification_model(generator):
    gc_model = GCNSupervisedGraphClassification(
        layer_sizes=[64, 64],
        activations=["relu", "relu"],
        generator=generator,
        dropout=0.5,
    )
    x_inp, x_out = gc_model.in_out_tensors()
    predictions = Dense(units=32, activation="relu")(x_out)
    predictions = Dense(units=16, activation="relu")(predictions)
    predictions = Dense(units=1, activation="sigmoid")(predictions)

    # Let's create the Keras model and prepare it for training
    model = Model(inputs=x_inp, outputs=predictions)
    model.compile(optimizer=Adam(0.005), loss=binary_crossentropy, metrics=["acc"])

    return model

In [ ]:
epochs = 200  # maximum number of training epochs
# folds = 10  # the number of folds for k-fold cross validation
# n_repeats = 5  # the number of repeats for repeated k-fold cross validation

In [ ]:
es = EarlyStopping(
    monitor="val_loss", min_delta=0, patience=25, restore_best_weights=True
)

In [ ]:
# To train in folds
def train_fold(model, train_gen, test_gen, es, epochs):
    history = model.fit(
        train_gen, epochs=epochs, validation_data=test_gen, verbose=0, callbacks=[es],
    )
    # calculate performance on the test data and return along with history
    test_metrics = model.evaluate(test_gen, verbose=0)
    test_acc = test_metrics[model.metrics_names.index("acc")]

    return history, test_acc

In [ ]:
# To train in folds
def get_generators(train_index, test_index, graph_labels, batch_size):
    train_gen = generator.flow(
        train_index, targets=graph_labels.iloc[train_index].values, weighted=True, batch_size=batch_size, shuffle=False, seed=42
    )
    test_gen = generator.flow(
        test_index, targets=graph_labels.iloc[test_index].values, weighted=True, batch_size=batch_size, shuffle=False, seed=42
    )

    return train_gen, test_gen

In [ ]:
# # To train in folds
# test_accs = []

# stratified_folds = model_selection.RepeatedStratifiedKFold(
#     n_splits=folds, n_repeats=n_repeats
# ).split(graph_labels, graph_labels)

# for i, (train_index, test_index) in enumerate(stratified_folds):
#     print(f"Training and evaluating on fold {i+1} out of {folds * n_repeats}...")
#     train_gen, test_gen = get_generators(
#         train_index, test_index, graph_labels, batch_size=30
#     )

#     model = create_graph_classification_model(generator)

#     history, acc = train_fold(model, train_gen, test_gen, es, epochs)

#     test_accs.append(acc)

In [ ]:
# To train in folds
test_accs = []

# stratified_folds = model_selection.RepeatedStratifiedKFold(
#     n_splits=folds, n_repeats=n_repeats
# ).split(graph_labels, graph_labels)

# for i, (train_index, test_index) in enumerate(stratified_folds):
for i in range(50):
    print(f"Training and evaluating on fold {i+1}...")
    
    train_gen, test_gen = get_generators(
        train_index, test_index, graph_labels, batch_size=30
    )

    model = create_graph_classification_model(generator)

    history, acc = train_fold(model, train_gen, test_gen, es, epochs)

    test_accs.append(acc)

In [ ]:
# 1: Accuracy over all folds mean: 76.0% and std: 1.8%
# 2: Accuracy over all folds mean: 75.7% and std: 4.7%
# 3: Accuracy over all folds mean: 73.8% and std: 1.9%
# 4: Accuracy over all folds mean: 76.2% and std: 4.2%
# 5: Accuracy over all folds mean: 70.3% and std: 5.3%
# 6: Accuracy over all folds mean: 74.0% and std: 5.2%

#---------- After setting seed
# 1: Accuracy over all folds mean: 73.5% and std: 6.7%
# 2: Accuracy over all folds mean: 73.3% and std: 3.9%
# 3: Accuracy over all folds mean: 74.7% and std: 3.9%
# 4: Accuracy over all folds mean: 74.4% and std: 2.0%
# 5: Accuracy over all folds mean: 74.2% and std: 1.6%
# 6: Accuracy over all folds mean: 74.3% and std: 2.4%

# Sepsis
# 1: Accuracy over all folds mean: 76.3% and std: 2.4%
print(
    f"Accuracy over all folds mean: {np.mean(test_accs)*100:.3}% and std: {np.std(test_accs)*100:.2}%"
)

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(test_accs)
plt.xlabel("Accuracy")
plt.ylabel("Count")

In [ ]:
# 2nd try
print(
    f"Accuracy over all folds mean: {np.mean(test_accs)*100:.3}% and std: {np.std(test_accs)*100:.2}%"
)

In [ ]:
plt.figure(figsize=(8, 6))
plt.hist(test_accs)
plt.xlabel("Accuracy")
plt.ylabel("Count")

In [ ]:
# # # shuffle the data
# import random

# c = list(zip(graphs, labels))
# random.shuffle(c)

# graphs, labels = zip(*c)

In [ ]:
# stratified_folds = model_selection.RepeatedStratifiedKFold(
#     n_splits=10, n_repeats=5
# ).split(graph_labels, graph_labels)

# for i, (train_index, test_index) in enumerate(stratified_folds):
#     print(f"\ntrain_index: \n{train_index}\ntest_index: \n{test_index}\n")
#     break
# print(f"train: {len(train_index)}, test: {len(test_index)}")

In [ ]:
type(graph_labels.iloc[train_index].values)

In [ ]:
graph_labels.iloc[test_index].values

In [ ]:
train_index.shape

In [ ]:
train_gen = generator.flow(
        train_index, targets=graph_labels.iloc[train_index].values, batch_size=40, shuffle=False, seed=42)
test_gen = generator.flow(
    test_index, targets=graph_labels.iloc[test_index].values, batch_size=40, shuffle=False, seed=42)

In [ ]:
model = create_graph_classification_model(generator)

history = model.fit(
        train_gen, epochs=200, validation_data=test_gen, verbose=1, callbacks=[es])

In [ ]:
# calculate performance on the test data and return along with history
# Node features all 1, edge features all 1
test_metrics = model.evaluate(test_gen, verbose=1)
test_acc = test_metrics[model.metrics_names.index("acc")]

In [ ]:
# not_weighted
print(test_metrics)
#test loss, test acc: [0.5977194905281067, 0.6794871687889099]
# 2nd try: [0.6099531650543213, 0.7435897588729858]
# 3: [0.6134859919548035, 0.692307710647583]
# 4: [0.5708641409873962, 0.7564102411270142]
print(test_acc)

In [ ]:
print("Generate predictions for samples")
predictions = model.predict(test_gen, verbose=0)
print("predictions shape:", predictions.shape)
# predictions shape: (79, 1)
print(predictions)

[[0.9633641 ]
 [0.9572478 ]
 [0.92709166]
 [0.67149305]
 [0.75392646]
 [0.80909836]
 [0.8878052 ]
 [0.35882965]
 [0.8740735 ]
 [0.9488548 ]
 [0.71857995]
 [0.8417202 ]
 [0.77317935]
 [0.97507536]
 [0.7930052 ]
 [0.3096424 ]
 [0.81946105]
 [0.3233545 ]
 [0.75175   ]
 [0.79594046]
 [0.2881279 ]
 [0.42757368]
 [0.35783005]
 [0.53080255]
 [0.42272243]
 [0.8731016 ]
 [0.2651747 ]
 [0.9176288 ]
 [0.8412247 ]
 [0.84611094]
 [0.899879  ]
 [0.4730718 ]
 [0.6183011 ]
 [0.8530265 ]
 [0.38916534]
 [0.41842264]
 [0.90864706]
 [0.82059276]
 [0.89827585]
 [0.31872994]
 [0.9104121 ]
 [0.8878981 ]
 [0.4407365 ]
 [0.6909543 ]
 [0.75392646]
 [0.88234955]
 [0.75392646]
 [0.75392646]
 [0.2803172 ]
 [0.8783453 ]
 [0.6334169 ]
 [0.8953688 ]
 [0.9676844 ]
 [0.46171212]
 [0.42416874]
 [0.7234482 ]
 [0.93954104]
 [0.7739153 ]
 [0.37732255]
 [0.7243462 ]
 [0.975207  ]
 [0.39652586]
 [0.6909691 ]
 [0.75392646]
 [0.3582555 ]
 [0.75392646]
 [0.94787574]
 [0.47389334]
 [0.9733413 ]
 [0.75392646]
 [0.931447  ]
 [0.53550804]
 [0.31291372]
 [0.3546905 ]
 [0.75392646]
 [0.75392646]
 [0.3483727 ]
 [0.7502146 ]]

In [ ]:
print(test_gen.targets)

In [ ]:
print(temp_test)

In [ ]:
temp_test = np.array(test_gen.targets)

In [ ]:
print(y_pred)

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

y_test = np.argmax(temp_test, axis=1) # Convert one-hot to index
y_pred = model.predict(test_gen)

print(classification_report(y_test, y_pred))